In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, make_scorer

import warnings
warnings.filterwarnings('ignore')

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
models = {'Linear regression': LinearRegression(),
          'Ridge': Ridge(),
          'Lasso': Lasso()}
metrics = {}

for method, model in models.items():
    model.fit(X_train, y_train)
    metrics[method] = [model.score(X_test, y_test)]
pd.DataFrame(metrics).T.rename(columns={0:'R2-score'}).sort_values(by='R2-score', ascending=False)

,R2-score
Linear regression,0.668759
Lasso,0.667145
Ridge,0.666222


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [6]:
alphas = np.array([10**i for i in range(-5, 6)])
models = {'Linear regression': LinearRegression(),
          'Ridge': RidgeCV(alphas=alphas),
          'Lasso': LassoCV(alphas=alphas)}
metrics = {}

for method, model in models.items():
    model.fit(X_train, y_train)
    metrics[method] = [model.score(X_test, y_test)]
pd.DataFrame(metrics).T.rename(columns={0:'R2-score'}).sort_values(by='R2-score', ascending=False)

,R2-score
Lasso,0.668760
Linear regression,0.668759
Ridge,0.668751


Подбор коэффициента регуляризации привел к увеличению коэффициента детерминации для соответствующих моделей

4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [7]:
# StandardScaler

models = {'Linear regression': LinearRegression(),
          'Ridge': Ridge(),
          'Lasso': Lasso()}
metrics = {}

for method, model in models.items():
    pipe = Pipeline([('StandardScaler', StandardScaler()), (method, model)])
    pipe.fit(X_train, y_train)
    metrics[method] = [pipe.score(X_test, y_test)]
pd.DataFrame(metrics).T.rename(columns={0:'R2-score'}).sort_values(by='R2-score', ascending=False)

,R2-score
Linear regression,0.668759
Ridge,0.668462
Lasso,0.623943


In [8]:
# MinMaxScaler

models = {'Linear regression': LinearRegression(),
          'Ridge': Ridge(),
          'Lasso': Lasso()}
metrics = {}

for method, model in models.items():
    pipe = Pipeline([('MinMaxScaler', MinMaxScaler()), (method, model)])
    pipe.fit(X_train, y_train)
    metrics[method] = [pipe.score(X_test, y_test)]
pd.DataFrame(metrics).T.rename(columns={0:'R2-score'}).sort_values(by='R2-score', ascending=False)

,R2-score
Ridge,0.676410
Linear regression,0.668759
Lasso,0.257392


Масштабирование признаков привело к росту метрики качества для обычной и Ridge регрессии, и снижению метрики в случау Lasso регрессии

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [9]:
# StandardScaler

alphas = np.array([10**i for i in range(-5, 6)])
models = {'Linear regression': LinearRegression(),
          'Ridge': RidgeCV(alphas=alphas),
          'Lasso': LassoCV(alphas=alphas)}
metrics = {}

for method, model in models.items():
    pipe = Pipeline([('StandardScaler', StandardScaler()), (method, model)])
    pipe.fit(X_train, y_train)
    metrics[method] = [pipe.score(X_test, y_test)]
pd.DataFrame(metrics).T.rename(columns={0:'R2-score'}).sort_values(by='R2-score', ascending=False)

,R2-score
Linear regression,0.668759
Lasso,0.668759
Ridge,0.665968


In [10]:
# MinMaxScaler

alphas = np.array([10**i for i in range(-5, 6)])
models = {'Linear regression': LinearRegression(),
          'Ridge': RidgeCV(alphas=alphas),
          'Lasso': LassoCV(alphas=alphas)}
metrics = {}

for method, model in models.items():
    pipe = Pipeline([('MinMaxScaler', MinMaxScaler()), (method, model)])
    pipe.fit(X_train, y_train)
    metrics[method] = [pipe.score(X_test, y_test)]
pd.DataFrame(metrics).T.rename(columns={0:'R2-score'}).sort_values(by='R2-score', ascending=False)

,R2-score
Ridge,0.670031
Lasso,0.668761
Linear regression,0.668759


Масштабирование признаков совместно с подбором коэффициента регуляризации в общем случае привело к росту метрики качества

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [11]:
# StandardScaler

models = {'Linear regression': LinearRegression(),
          'Ridge': Ridge(),
          'Lasso': Lasso()}
metrics = {}

for method, model in models.items():
    pipe = Pipeline([('PolynomialFeatures', PolynomialFeatures(2)), ('StandardScaler', StandardScaler()), (method, model)])
    pipe.fit(X_train, y_train)
    metrics[method] = [pipe.score(X_test, y_test)]
pd.DataFrame(metrics).T.rename(columns={0:'R2-score'}).sort_values(by='R2-score', ascending=False)

,R2-score
Ridge,0.847179
Linear regression,0.805583
Lasso,0.685565


In [12]:
# MinMaxScaler

models = {'Linear regression': LinearRegression(),
          'Ridge': Ridge(),
          'Lasso': Lasso()}
metrics = {}

for method, model in models.items():
    pipe = Pipeline([('PolynomialFeatures', PolynomialFeatures(2)), ('MinMaxScaler', MinMaxScaler()), (method, model)])
    pipe.fit(X_train, y_train)
    metrics[method] = [pipe.score(X_test, y_test)]
pd.DataFrame(metrics).T.rename(columns={0:'R2-score'}).sort_values(by='R2-score', ascending=False)

,R2-score
Linear regression,0.805583
Ridge,0.798400
Lasso,0.267968


Добавление полиномиальных признаков в общес случае привело к росту R2-score, за исключением Lasso регрессии - требуется подбор коэффициента регуляризации

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [13]:
# Не самый удачный способ
alphas = np.array([10**i for i in range(-5, 6)])
scalers = [('StandardScaler', StandardScaler()), ('MinMaxScaler', MinMaxScaler())]
poly = [(f'PolynomialFeatures_{i}', PolynomialFeatures(i)) for i in range(1, 5)]
models = [('Linear regression', LinearRegression()),
          ('Ridge', RidgeCV(alphas=alphas)),
          ('Lasso', LassoCV(alphas=alphas))]

import itertools
combinations = []
for comb in itertools.combinations(itertools.chain(scalers, poly, models), 3):
    counter_models = 0
    counter_scalers = 0
    counter_poly = 0
    for element in comb:
        if element in models:
            counter_models += 1
        elif element in scalers:
            counter_scalers += 1
        elif element in poly:
            counter_poly += 1
    if counter_poly == 1 and counter_models == 1 and counter_scalers== 1:
        combinations.append(comb)

In [14]:
params = {'steps': combinations}
model = GridSearchCV(estimator=Pipeline(steps=combinations[0]), param_grid=params, n_jobs=16)
model.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=(('StandardScaler', StandardScaler()),
                                       ('PolynomialFeatures_1',
                                        PolynomialFeatures(degree=1)),
                                       ('Linear regression',
                                        LinearRegression()))),
             n_jobs=16,
             param_grid={'steps': [(('StandardScaler', StandardScaler()),
                                    ('PolynomialFeatures_1',
                                     PolynomialFeatures(degree=1)),
                                    ('Linear regression', LinearRegression())),
                                   (('StandardScale...
                                    ('PolynomialFeatures_4',
                                     PolynomialFeatures(degree=4)),
                                    ('Ridge',
                                     RidgeCV(alphas=array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
    

In [15]:
print(f'Лучшая комбинация:')
print(f'1. Регуляризация: {model.best_estimator_[2]}')
print(f'2. Коэффициент регуляризации: {model.best_estimator_[2].alpha_}')
print(f'3. Масштабирование: {model.best_estimator_[0]}')
print(f'4. Степень полинома: {model.best_estimator_[1]}')
print(f'\nR2-score: {model.score(X_test, y_test)}')

Лучшая комбинация:
1. Регуляризация: LassoCV(alphas=array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]))
2. Коэффициент регуляризации: 0.001
3. Масштабирование: MinMaxScaler()
4. Степень полинома: PolynomialFeatures(degree=3)

R2-score: 0.8449734005131505


In [16]:
pipe = Pipeline(steps=[('PolynomialFeatures', PolynomialFeatures(degree=1)), ('Scaler', StandardScaler()), ('Regressor', LinearRegression())])

scalers = [StandardScaler(), MinMaxScaler()]
models = [Ridge(), Lasso()]

params = {'Scaler': scalers,
          'Regressor': models,
          'PolynomialFeatures__degree': [1, 2, 3, 4],
          'Regressor__alpha':  [0] + [10**i for i in range(-5, 6)],
          }

model = GridSearchCV(estimator=pipe, param_grid=params, n_jobs=16)
model.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('PolynomialFeatures',
                                        PolynomialFeatures(degree=1)),
                                       ('Scaler', StandardScaler()),
                                       ('Regressor', LinearRegression())]),
             n_jobs=16,
             param_grid={'PolynomialFeatures__degree': [1, 2, 3, 4],
                         'Regressor': [Ridge(), Lasso(alpha=0.001)],
                         'Regressor__alpha': [0, 1e-05, 0.0001, 0.001, 0.01,
                                              0.1, 1, 10, 100, 1000, 10000,
                                              100000],
                         'Scaler': [StandardScaler(), MinMaxScaler()]})

In [17]:
print(f'Лучшая комбинация:')
print(f'1. Регуляризация: {model.best_estimator_[2]}')
print(f'2. Коэффициент регуляризации: {model.best_estimator_[2].alpha}')
print(f'3. Масштабирование: {model.best_estimator_[1]}')
print(f'4. Степень полинома: {model.best_estimator_[0].degree}')
print(f'\nR2-score: {model.score(X_test, y_test)}')

Лучшая комбинация:
1. Регуляризация: Lasso(alpha=0.001)
2. Коэффициент регуляризации: 0.001
3. Масштабирование: MinMaxScaler()
4. Степень полинома: 3

R2-score: 0.8651767295811961


В данном эксперименте наблюдается наибольший коэффициент детерминации.

http://archive.ics.uci.edu/ml/datasets/Adult

In [18]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [19]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [20]:
y = data[14].replace({'<=50K': 0, '>50K': 1})
X = data.drop(columns=[14])

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [21]:
# В явном виде пропусков не обнаружено
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       48842 non-null  int64 
 1   1       48842 non-null  object
 2   2       48842 non-null  int64 
 3   3       48842 non-null  object
 4   4       48842 non-null  int64 
 5   5       48842 non-null  object
 6   6       48842 non-null  object
 7   7       48842 non-null  object
 8   8       48842 non-null  object
 9   9       48842 non-null  object
 10  10      48842 non-null  int64 
 11  11      48842 non-null  int64 
 12  12      48842 non-null  int64 
 13  13      48842 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


In [22]:
# '?' считаем за пропуск
X[1].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [23]:
# Перевел шаг в pipeline

# imp_freq = SimpleImputer(missing_values='?', strategy='most_frequent')
# X = imp_freq.fit_transform(X)

10. Выберите колонки с числовыми и категориальными переменными.

In [24]:
numeric_features = []
categorical_features = []
for column in data.columns:
    if column == 14:
        continue
    if data[column].dtype == np.dtype('int64'):
        numeric_features.append(column)
    elif data[column].dtype == np.dtype('object_'):
        categorical_features.append(column)
    else:
        raise Exception('Check data types')

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [25]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), numeric_features),
        ("cat", OneHotEncoder(), categorical_features),
    ]
)

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [26]:
y.value_counts()
#accuracy_score, f1_score

0    37155
1    11687
Name: 14, dtype: int64

In [27]:
print('Constant prediction')
print(f'Accuracy: {accuracy_score(y, np.array([0 for _ in range(len(y))]))}')
print(f'F1-score: {f1_score(y, np.array([0 for _ in range(len(y))]))}')

Constant prediction
Accuracy: 0.7607182343065395
F1-score: 0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [28]:
models = {'LogisticRegression': LogisticRegression(),
          #'SVC': SVC(),  # долгий расчет
          'LinearSVC': LinearSVC()}
for model_name, model in models.items():
    res = cross_validate(estimator=Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                                   ('preprocessor', preprocessor),
                                                   ('clf', model)]),
                         X=X,
                         y=y,
                         cv=5,
                         scoring ={'accuracy': make_scorer(accuracy_score),
                                   'f1': make_scorer(f1_score)})
    print(model_name)
    print(f"Accuracy: {np.mean(res['test_accuracy'][~np.isnan(res['test_accuracy'])])}")
    print(f"F1-score: {np.mean(res['test_f1'][~np.isnan(res['test_f1'])])}\n")

LogisticRegression
Accuracy: 0.8497978649766191
F1-score: 0.6540050902259514

LinearSVC
Accuracy: 0.8518196579730514
F1-score: 0.656125117729116



Модели показали лучший результат относительно константного предсказания

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [29]:
imputer = SimpleImputer(missing_values='?', strategy='most_frequent')

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [30]:
models = {'LogisticRegression': LogisticRegression(),
          #'SVC': SVC(),  # долгий расчет
          'LinearSVC': LinearSVC()}
for model_name, model in models.items():
    res = cross_validate(estimator=Pipeline(steps=[('imputer', SimpleImputer(missing_values='?', strategy='most_frequent')),
                                                   ('preprocessor', preprocessor),
                                                   ('clf', model)]),
                         X=X,
                         y=y,
                         cv=5,
                         scoring ={'accuracy': make_scorer(accuracy_score),
                                   'f1': make_scorer(f1_score)})
    print(model_name)
    print(f"Accuracy: {np.mean(res['test_accuracy'][~np.isnan(res['test_accuracy'])])}")
    print(f"F1-score: {np.mean(res['test_f1'][~np.isnan(res['test_f1'])])}\n")

LogisticRegression
Accuracy: 0.849593099052486
F1-score: 0.6523707327202056

LinearSVC
Accuracy: 0.8505144461549927
F1-score: 0.6505724715080432



Результат ухудшился

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [31]:
data = data.replace({'?', None}).dropna()
y = data[14].replace({'<=50K': 0, '>50K': 1})
X = data.drop(columns=[14])

models = {'LogisticRegression': LogisticRegression(),
          # SVC': SVC(),  # долгий расчет
          'LinearSVC': LinearSVC()}
for model_name, model in models.items():
    res = cross_validate(estimator=Pipeline(steps=[('imputer', SimpleImputer(missing_values='?', strategy='most_frequent')),
                                                   ('preprocessor', preprocessor),
                                                   ('clf', model)]),
                         X=X,
                         y=y,
                         cv=5,
                         scoring ={'accuracy': make_scorer(accuracy_score),
                                   'f1': make_scorer(f1_score)})
    print(model_name)
    print(f"Accuracy: {np.mean(res['test_accuracy'][~np.isnan(res['test_accuracy'])])}")
    print(f"F1-score: {np.mean(res['test_f1'][~np.isnan(res['test_f1'])])}\n")

LogisticRegression
Accuracy: 0.849593099052486
F1-score: 0.6523707327202056

LinearSVC
Accuracy: 0.8505144461549927
F1-score: 0.6505724715080432



Результат практически не изменился

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [32]:
data = pd.read_csv(link, header=None)
y = data[14].replace({'<=50K': 0, '>50K': 1})
X = data.drop(columns=[14])

models = {'RF': RandomForestClassifier(n_jobs=16),
          'GBC': CatBoostClassifier(thread_count=16, verbose=0)}
for model_name, model in models.items():
    res = cross_validate(estimator=Pipeline(steps=[('imputer', SimpleImputer(missing_values='?', strategy='most_frequent')),
                                                   ('preprocessor', preprocessor),
                                                   ('clf', model)]),
                         X=X,
                         y=y,
                         cv=5,
                         scoring ={'accuracy': make_scorer(accuracy_score),
                                   'f1': make_scorer(f1_score)})
    print(model_name)
    print(f"Accuracy: {np.mean(res['test_accuracy'][~np.isnan(res['test_accuracy'])])}")
    print(f"F1-score: {np.mean(res['test_f1'][~np.isnan(res['test_f1'])])}\n")

RF
Accuracy: 0.8504888078411469
F1-score: 0.6640337734167814

GBC
Accuracy: 0.8724216334258303
F1-score: 0.7095762338156018



Для рассмотренных алгоритмов метрики качества оказались выше. Причем модель градиентного бустинга на решающих деревьях показала себя лучше модели случайного леса. Возможно, для максимизации метрики качества оптимальнее использовать бустинговые модели (в ущерб интерпретируемости).

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [36]:
data = pd.read_csv(link, header=None)
y = data[14].replace({'<=50K': 0, '>50K': 1})
X = data.drop(columns=[14])

categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')), ('OHE', OneHotEncoder())])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

pipe = Pipeline(steps=[('preprocessor', preprocessor), ('clf', LogisticRegression())])
params = {
    'preprocessor__cat__imputer__missing_values': [np.nan, '?'],
    'preprocessor__num': [StandardScaler(), MinMaxScaler()],
    'clf': [LogisticRegression(n_jobs=16), LinearSVC(), RandomForestClassifier(n_jobs=16), CatBoostClassifier(thread_count=16, verbose=False)]
}

In [37]:
model = GridSearchCV(estimator=pipe,
                     param_grid=params,
                     cv=5,
                     n_jobs=16,
                     scoring={'accuracy': make_scorer(accuracy_score),
                              'f1': make_scorer(f1_score)},
                     refit='f1'
                     )
model.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         MinMaxScaler(),
                                                                         [0, 2,
                                                                          4, 10,
                                                                          11,
                                                                          12]),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('OHE',
                           

In [38]:
model.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  [0, 2, 4, 10, 11, 12]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OHE',
                                                                   OneHotEncoder())]),
                                                  [1, 3, 5, 6, 7, 8, 9, 13])])),
                ('clf', LogisticRegression(n_jobs=16))])

In [53]:
f1_scores = np.array([model.cv_results_[f'split{i}_test_f1'][0] for i in range(5)])
accuracy_scores = np.array([model.cv_results_[f'split{i}_test_accuracy'][0] for i in range(5)])
print(f"Accuracy: {np.mean(f1_scores[~np.isnan(f1_scores)])}")
print(f"F1-score: {np.mean(accuracy_scores[~np.isnan(accuracy_scores)])}\n")

Accuracy: 0.6589598754784799
F1-score: 0.8512054309378754

